Завдання 2

In [44]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from google.colab import files
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
data = files.upload()
df = pd.read_csv(next(iter(data)))

Saving titanic.csv to titanic (6).csv


In [45]:
#1. Проводимо попередній аналіз
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [46]:
df.shape

(418, 12)

In [47]:
df.dtypes

,0
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [48]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1


In [49]:
df = df.fillna(df.mean(numeric_only=True), axis=0)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [50]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


In [51]:
df = df.drop(columns=['Cabin'])
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [52]:
selected_columns = ['Survived', 'Pclass', 'Sex', 'Age', 'Fare']
df1 = df[selected_columns]
df1.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,34.5,7.8292
1,1,3,female,47.0,7.0000
2,0,2,male,62.0,9.6875
3,0,3,male,27.0,8.6625
4,1,3,female,22.0,12.2875


In [53]:
unique_values = df1['Sex'].unique()
print("Унікальні значення в стовпці 'Sex':", unique_values)
df1['Sex'] = df1['Sex'].map({'female': 0, 'male': 1})
df1.head()

Унікальні значення в стовпці 'Sex': ['male' 'female']


<ipython-input-53-8092da5b217a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Sex'] = df1['Sex'].map({'female': 0, 'male': 1})


,Survived,Pclass,Sex,Age,Fare
0,0,3,1,34.5,7.8292
1,1,3,0,47.0,7.0000
2,0,2,1,62.0,9.6875
3,0,3,1,27.0,8.6625
4,1,3,0,22.0,12.2875


In [54]:
df1.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,0
Fare,0


In [55]:
df1 = df1.astype(float)

In [56]:
df1.dtypes

,0
Survived,float64
Pclass,float64
Sex,float64
Age,float64
Fare,float64


In [58]:
#Будуємо 3 класифікатори на вибір, визначаємо оптимальні параметри (оптимізатор – на вибір).
X = df1[['Pclass', 'Sex', 'Age', 'Fare']]
y = df1['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [59]:
#Побудуйте три моделі класифікації (на власний вибір).
# 1. Логістична регресія
logreg = LogisticRegression(max_iter=1000)
logreg_params = {
    'C': [0.1, 1, 10],
    'penalty': ['l2']
}

logreg_grid = GridSearchCV(logreg, logreg_params, cv=5)
logreg_grid.fit(X_train, y_train)
logreg_best = logreg_grid.best_estimator_

# 2. Дерево рішень
tree = DecisionTreeClassifier(random_state=42)
tree_params = {
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10]
}

tree_grid = GridSearchCV(tree, tree_params, cv=5)
tree_grid.fit(X_train, y_train)
tree_best = tree_grid.best_estimator_

# 3. Метод опорних векторів (SVM)
svm = SVC()
svm_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

svm_grid = GridSearchCV(svm, svm_params, cv=5)
svm_grid.fit(X_train, y_train)
svm_best = svm_grid.best_estimator_

# Оцінка моделей
models = {'Logistic Regression': logreg_best, 'Decision Tree': tree_best, 'SVM': svm_best}

for model_name, model in models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")

Logistic Regression Accuracy: 1.0
Decision Tree Accuracy: 1.0
SVM Accuracy: 1.0


In [60]:
#Підберіть оптимальні параметри для кожної моделі
best_params = {
    'Model': ['Logistic Regression', 'Decision Tree', 'SVM'],
    'Best Parameters': [
        logreg_grid.best_params_,
        tree_grid.best_params_,
        svm_grid.best_params_
    ]
}
best_params_df = pd.DataFrame(best_params)
best_params_df

,Model,Best Parameters
0,Logistic Regression,"{'C': 0.1, 'penalty': 'l2'}"
1,Decision Tree,"{'max_depth': 3, 'min_samples_split': 2}"
2,SVM,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}"


In [62]:
for model_name, model in models.items():
    print(f"Model: {model_name}")
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Точність (accuracy): {accuracy:.2f}")
    class_report = classification_report(y_test, y_pred)
    print("Звіт про класифікацію (classification report):")
    print(class_report)
    print('-' * 60)


Model: Logistic Regression
Точність (accuracy): 1.00
Звіт про класифікацію (classification report):
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        85
         1.0       1.00      1.00      1.00        41

    accuracy                           1.00       126
   macro avg       1.00      1.00      1.00       126
weighted avg       1.00      1.00      1.00       126

------------------------------------------------------------
Model: Decision Tree
Точність (accuracy): 1.00
Звіт про класифікацію (classification report):
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        85
         1.0       1.00      1.00      1.00        41

    accuracy                           1.00       126
   macro avg       1.00      1.00      1.00       126
weighted avg       1.00      1.00      1.00       126

------------------------------------------------------------
Model: SVM
Точність (accuracy)

In [68]:
#Робимо прогноз для 10 випадків.
best_params = {'C': 0.1, 'penalty': 'l2'}
logreg = LogisticRegression(**best_params, max_iter=1000)
logreg.fit(X_train, y_train)
np.random.seed(42)
random_indices = np.random.choice(X_test.shape[0], 10, replace=False)
X_random = X_test[random_indices]
y_random_actual = y_test.iloc[random_indices]
y_random_pred = logreg.predict(X_random)

comparison_df = pd.DataFrame({
    'Sample Index': random_indices,
    'Real Class': y_random_actual.values,
    'Predicted Class': y_random_pred
})

comparison_df

,Sample Index,Real Class,Predicted Class
0,73,1.0,1.0
1,19,0.0,0.0
2,116,1.0,1.0
3,67,0.0,0.0
4,94,0.0,0.0
5,77,1.0,1.0
6,31,1.0,1.0
7,53,0.0,0.0
8,117,0.0,0.0
9,44,0.0,0.0


Висновок.
Провели попередній аналіз даних. Розділили дані на навчальний і тестовий набори. Побудували три моделі класифікації: логістична регресія, дерево рішень та метод опорних векторів (SVM). Підібрали оптимальні параметри для кожної моделі.Всі моделі мають точніть 1.